# Hyperparameter Tuning for xgboost model 

### 1. Grid Search
### 2. Random Search
### 3. Bayesian Optimization
### 4. Genetic Algorithm

In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 
import datetime as dt


from sklearn.datasets import load_diabetes
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold, RandomizedSearchCV,TimeSeriesSplit
import xgboost as xgb
from scipy.stats import uniform, randint
from GPyOpt.methods import BayesianOptimization
from tpot import TPOTRegressor
from pprint import pprint


### Data Preprocessing

In [2]:
df1=pd.read_csv('llama_forecast_train.csv')
df2=pd.read_csv('historical_weather.csv')
df2.rename(columns={'HOUR':'DAY'},inplace='True')
df1=pd.merge(df1,df2,on=['DAY'], how= "left")

In [3]:
# Getting date features from the training data
df1['Date']=pd.to_datetime(df1['DAY'])
df1.sort_values(by='Date',ascending=True,inplace=True)
df1['Year']=df1['Date'].dt.year
df1['Month']=df1['Date'].dt.month
df1['Week']=df1['Date'].dt.week
df1['Day']=df1['Date'].dt.day
df1['Week Day']=df1['Date'].dt.dayofweek
df1['Year Day']=df1['Date'].dt.dayofyear
df1.drop(['Date','DAY'],axis=1,inplace=True)

In [5]:
df3=pd.concat([df1, pd.get_dummies(df1[['HABITAT NAME']])], axis=1).drop(['HABITAT NAME'],axis=1)
df3=df3.reset_index(drop=True)
df3.head()

,HABITAT ID,AVAILABLE LLAMAS,TEMPERATURE,HUMIDITY,PRECIPITATION,WINDSPEED,Year,Month,Week,Day,...,HABITAT NAME_Peachtree,HABITAT NAME_Perfect Square,HABITAT NAME_Primary Cay,HABITAT NAME_Pulp Point,HABITAT NAME_Random Forest,HABITAT NAME_Ridge Road,HABITAT NAME_Shortest Path,HABITAT NAME_Tierra del Fuego,HABITAT NAME_Vector Field,HABITAT NAME_Wildcat Way
0,0,74,73,71,0.0,9.4,2017,7,26,1,...,1,0,0,0,0,0,0,0,0,0
1,18,58,73,71,0.0,9.4,2017,7,26,1,...,0,0,0,0,0,0,1,0,0,0
2,20,96,73,71,0.0,9.4,2017,7,26,1,...,0,0,0,0,0,0,0,0,1,0
3,22,88,73,71,0.0,9.4,2017,7,26,1,...,0,0,0,0,0,0,0,0,0,0
4,7,48,73,71,0.0,9.4,2017,7,26,1,...,0,1,0,0,0,0,0,0,0,0


### Model Building

In [9]:
y=df3['AVAILABLE LLAMAS']
x=df3.drop(['AVAILABLE LLAMAS'],axis=1)

In [16]:

import xgboost as xgb
model=xgb.XGBRegressor(objective ='reg:squarederror')

In [80]:

b=int(len(df1)*.75)
x_train, x_test=x.iloc[0:b, :], x.iloc[b:-1, :]
y_train, y_test=y.iloc[0:b], y.iloc[b:-1]

model.fit(x_train,y_train)

#Function to calculate rmse after doing hyperparameter optimization

def result(model):
    y_pred=model.predict(x_test)
    mse=mean_squared_error(y_test,y_pred)
    rmse=np.sqrt(mse)
    return rmse
    
    
result(model)

23.41276388732124

## Hyperparameter Tuning

In [46]:
#Splitting data for hyperparameter tuning
tss=TimeSeriesSplit(n_splits=3)


### 1.Grid Search

In [19]:
pprint(model.get_xgb_params())

{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'gamma': 0,
 'gpu_id': -1,
 'interaction_constraints': '',
 'learning_rate': 0.300000012,
 'max_delta_step': 0,
 'max_depth': 6,
 'min_child_weight': 1,
 'monotone_constraints': '()',
 'n_jobs': 0,
 'num_parallel_tree': 1,
 'objective': 'reg:squarederror',
 'random_state': 0,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'subsample': 1,
 'tree_method': 'exact',
 'validate_parameters': 1,
 'verbosity': None}


In [22]:
# define the grid
params = {
    "learning_rate": [0.01, 0.1],
    "max_depth": [2, 4, 6],
    "n_estimators": [10, 100,150],
    "subsample": [0.8, 1],
    "min_child_weight": [1, 3],
    "reg_lambda": [1, 3],
    "reg_alpha": [1, 3]
}

# setup the grid search
grid_search = GridSearchCV(model,
                           param_grid=params,
                           cv=tss,
                           verbose=1,
                           n_jobs=1,
                           return_train_score=True)
grid_search.fit(x_train, y_train)

Fitting 3 folds for each of 288 candidates, totalling 864 fits


GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bynode=1,
                                    colsample_bytree=1, gamma=0, gpu_id=-1,
                                    importance_type='gain',
                                    interaction_constraints='',
                                    learning_rate=0.300000012, max_delta_step=0,
                                    max_depth=6, min_child_weight=1,
                                    missing=na...
                                    num_parallel_tree=1, random_state=0,
                                    reg_alpha=0, reg_lambda=1,
                                    scale_pos_weight=1, subsample=1,
                                    tree_method='exact', validate_parameters=1,
                                    verbosity=None),
             n_jobs=1,
   

In [23]:
grid_search.best_params_

{'learning_rate': 0.1,
 'max_depth': 6,
 'min_child_weight': 1,
 'n_estimators': 100,
 'reg_alpha': 3,
 'reg_lambda': 3,
 'subsample': 0.8}

In [81]:
result(grid_search)

22.389818301906804

### 2. Random Search

In [31]:

params = {
    "learning_rate": [0.001, 0.01, 0.1, 0.5, 1.],
    "max_depth": randint(1, 30),
    "n_estimators": randint(10, 250),
    "subsample": uniform(0.05, 0.95),  # so uniform on [.05,.05+.95] = [.05,1.]
    "min_child_weight": randint(1, 30),
    "reg_alpha": uniform(0, 10),
    "reg_lambda": uniform(0, 10)
}

random_search = RandomizedSearchCV(
    model,
    param_distributions=params,
    random_state=8675309,
    n_iter=200,
    cv=tss,
    verbose=1,
    n_jobs=1,
    return_train_score=True)

random_search.fit(x_train, y_train)

Fitting 3 folds for each of 200 candidates, totalling 600 fits


RandomizedSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                   estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                          colsample_bylevel=1,
                                          colsample_bynode=1,
                                          colsample_bytree=1, gamma=0,
                                          gpu_id=-1, importance_type='gain',
                                          interaction_constraints='',
                                          learning_rate=0.300000012,
                                          max_delta_step=0, max_depth=6,
                                          min_child_weight=1, miss...
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000023621E25C48>,
                                        'reg_alpha': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000023621E311C8>,
             

In [32]:
random_search.best_params_

{'learning_rate': 0.5,
 'max_depth': 16,
 'min_child_weight': 4,
 'n_estimators': 59,
 'reg_alpha': 8.507781740418231,
 'reg_lambda': 7.697186827516576,
 'subsample': 0.37389972137581645}

In [82]:
result(random_search)

24.604058381553173

### 3.Bayesian Optimization

In [35]:
# unfold to see code
np.random.seed(8675309)  # seed courtesy of Tommy Tutone
# from GPyOpt.methods import BayesianOptimization
# from sklearn.model_selection import cross_val_score, KFold

hp_bounds = [{
    'name': 'learning_rate',
    'type': 'continuous',
    'domain': (0.001, 1.0)
}, {
    'name': 'max_depth',
    'type': 'discrete',
    'domain': (1, 10)
}, {
    'name': 'n_estimators',
    'type': 'discrete',
    'domain': (10, 150)
}, {
    'name': 'subsample',
    'type': 'continuous',
    'domain': (0.05, 1.0)
}, {
    'name': 'min_child_weight',
    'type': 'discrete',
    'domain': (1, 20)
}, {
    'name': 'reg_alpha',
    'type': 'continuous',
    'domain': (0, 5)
}, {
    'name': 'reg_lambda',
    'type': 'continuous',
    'domain': (0, 5)
}]


# Optimization objective
def cv_score(hyp_parameters):
    hyp_parameters = hyp_parameters[0]
    xgb_model = xgb.XGBRegressor(objective='reg:squarederror',
                                 learning_rate=hyp_parameters[0],
                                 max_depth=int(hyp_parameters[1]),
                                 n_estimators=int(hyp_parameters[2]),
                                 subsample=hyp_parameters[3],
                                 min_child_weight=int(hyp_parameters[4]),
                                 reg_alpha=hyp_parameters[5],
                                 reg_lambda=hyp_parameters[6])
    scores = cross_val_score(model,
                             X=x_train,
                             y=y_train,
                             cv=tss)
    return np.array(scores.mean())  # return average of 5-fold scores


optimizer = BayesianOptimization(f=cv_score,
                                 domain=hp_bounds,
                                 model_type='GP',
                                 acquisition_type='EI',
                                 acquisition_jitter=0.05,
                                 exact_feval=True,
                                 maximize=True,
                                 verbosity=True)

optimizer.run_optimization(max_iter=20,verbosity=True)

num acquisition: 1, time elapsed: 2.57s
num acquisition: 2, time elapsed: 5.19s
num acquisition: 3, time elapsed: 8.01s
num acquisition: 4, time elapsed: 10.54s
num acquisition: 5, time elapsed: 13.00s
num acquisition: 6, time elapsed: 15.61s
num acquisition: 7, time elapsed: 17.97s
num acquisition: 8, time elapsed: 20.35s
num acquisition: 9, time elapsed: 22.82s
num acquisition: 10, time elapsed: 24.76s
num acquisition: 11, time elapsed: 27.10s
num acquisition: 12, time elapsed: 29.60s
num acquisition: 13, time elapsed: 32.03s
num acquisition: 14, time elapsed: 34.71s
num acquisition: 15, time elapsed: 36.86s
num acquisition: 16, time elapsed: 39.47s
num acquisition: 17, time elapsed: 42.54s
num acquisition: 18, time elapsed: 45.65s
num acquisition: 19, time elapsed: 48.33s
num acquisition: 20, time elapsed: 50.93s


In [36]:
best_hyp_set = {}
for i in range(len(hp_bounds)):
    if hp_bounds[i]['type'] == 'continuous':
        best_hyp_set[hp_bounds[i]['name']] = optimizer.x_opt[i]
    else:
        best_hyp_set[hp_bounds[i]['name']] = int(optimizer.x_opt[i])
best_hyp_set

{'learning_rate': 0.35488033434114763,
 'max_depth': 10,
 'n_estimators': 10,
 'subsample': 0.5687129957781784,
 'min_child_weight': 1,
 'reg_alpha': 3.950325217852195,
 'reg_lambda': 0.2536401881692735}

In [83]:
bayopt_search = xgb.XGBRegressor(objective='reg:squarederror',**best_hyp_set)
bayopt_search.fit(x_train,y_train)
result(bayopt_search)

23.795493176431403

### 4. Genetic Algorithm from TPOT library

In [38]:
# from tpot import TPOTRegressor

tpot_config = {
    'xgboost.XGBRegressor': {
        'n_estimators': [100],
        'max_depth': range(1, 11),
        'learning_rate': np.append(np.array([.001,.01]),np.arange(0.05,1.05,.05)),
        'subsample': np.arange(0.05, 1.01, 0.05),
        'min_child_weight': range(1, 21),
        'reg_alpha': np.arange(1.0,5.25,.25),
        'reg_lambda': np.arange(1.0,5.25,.25),
        'nthread': [1],
        'objective': ['reg:squarederror']
    }
}

tpot = TPOTRegressor(scoring = 'r2',
                     generations=50,
                     population_size=20,
                     verbosity=2,
                     config_dict=tpot_config,
                     cv=tss,
                     template='Regressor', #no stacked models
                     random_state=8675309)

tpot.fit(x_train, y_train)


Generation 1 - Current best internal CV score: 0.8164448565204342

Generation 2 - Current best internal CV score: 0.8183018885515837

Generation 3 - Current best internal CV score: 0.8183018885515837

Generation 4 - Current best internal CV score: 0.8183018885515837

Generation 5 - Current best internal CV score: 0.818586428023087

Generation 6 - Current best internal CV score: 0.8201404698660323

Generation 7 - Current best internal CV score: 0.8201404698660323

Generation 8 - Current best internal CV score: 0.8201404698660323

Generation 9 - Current best internal CV score: 0.8201404698660323

Generation 10 - Current best internal CV score: 0.8201404698660323

Generation 11 - Current best internal CV score: 0.8201404698660323

Generation 12 - Current best internal CV score: 0.8201404698660323

Generation 13 - Current best internal CV score: 0.8244763501210377

Generation 14 - Current best internal CV score: 0.8244763501210377

Generation 15 - Current best internal CV score: 0.8244763

TPOTRegressor(config_dict={'xgboost.XGBRegressor': {'learning_rate': array([0.001, 0.01 , 0.05 , 0.1  , 0.15 , 0.2  , 0.25 , 0.3  , 0.35 ,
       0.4  , 0.45 , 0.5  , 0.55 , 0.6  , 0.65 , 0.7  , 0.75 , 0.8  ,
       0.85 , 0.9  , 0.95 , 1.   ]),
                                                    'max_depth': range(1, 11),
                                                    'min_child_weight': range(1, 21),
                                                    'n_estimators': [100],
                                                    'nthread': [1],
                                                    'objective': ['reg:squarederror'],
                                                    'reg_alpha': array([1.  , 1.25, 1.5 , 1.75, 2.  , 2.25, 2.5 , 2.7...
                                                    'reg_lambda': array([1.  , 1.25, 1.5 , 1.75, 2.  , 2.25, 2.5 , 2.75, 3.  , 3.25, 3.5 ,
       3.75, 4.  , 4.25, 4.5 , 4.75, 5.  ]),
                                                    's

In [84]:
result(tpot)

27.869062596452192

## Final Results

In [85]:
# initialize list of lists
data = [['Regular Model', result(model)], ['Grid Search', result(grid_search)], ['Random_Search', result(random_search)], ['Bayesian Optimization', result(bayopt_search)], ['Genetic Algorithm', result(tpot)]]
  
# Create the pandas DataFrame
df = pd.DataFrame(data, columns = ['Model', 'RMSE'])
  
# print dataframe.
df


,Model,RMSE
0,Regular Model,23.412764
1,Grid Search,22.389818
2,Random_Search,24.604058
3,Bayesian Optimization,23.795493
4,Genetic Algorithm,27.869063


## We can see Grid Seach gave use the best RMSE for this dataset. It is better to try all the approaches to see which one optimize the final accuracy !